# Merging state, county, and city polygons with landvote data


In [ ]:
import ibis
from ibis import _
import streamlit as st
import ibis.expr.datatypes as dt  

conn = ibis.duckdb.connect(extensions=["spatial"])

# Landvote data


In [ ]:
landvote = conn.read_csv("landvote.csv")

landvote_df = (landvote
                .rename(jurisdiction = "Jurisdiction Type", state = "State", name = "Jurisdiction Name")
                .rename(amount = 'Conservation Funds at Stake', yes = '% Yes')
                .mutate(amount_n=_.amount.replace('$', '').replace(',', '').cast('float'))
                .mutate(log_amount=_.amount_n.log())
                .mutate(year=_['Date'].year().cast('int32'))
                .mutate(
                    yes=ibis.case()
                        .when(_.yes.isin(['Pass', 'None','Fail']), None)  # Handle non-numeric cases
                        .when(_.yes.notnull(), (_.yes.replace('%', '').cast('float').round(2).cast(dt.float64)).cast(dt.string) + '%')  # Convert valid percentages and add %
                        .else_(None)  # Default to None for other cases
                        .end()
                )
                .mutate(
                    municipal=ibis.case()
                        .when(_.jurisdiction.isin(['State','County']), None) 
                        .else_(_.name) 
                        .end()
                )
                .mutate(
                    county=ibis.case()
                        .when(_.jurisdiction.isin(['State','Municipal']), None)  
                        .else_(_.name) 
                        .end()
                )
               .mutate(log_amount = _.log_amount.round(4))
               .select('name','state','county','municipal','jurisdiction','Status', 'yes', 'year', 'amount', 'log_amount', )
               )

# State Data

In [ ]:
state_boundaries = "https://data.source.coop/cboettig/us-boundaries/us-state-territory.parquet"
landvote_states = landvote_df.filter(_.jurisdiction == "State")

state = (conn
        .read_parquet(state_boundaries)
        .rename(state = "STUSPS", state_name = "NAME")
        .select('state','geometry','state_name')
        .cast({"geometry": "geometry"})
        )

df_state = (state
            .join(landvote_states, "state",how = "inner")
            .select('state','county','municipal','jurisdiction','geometry','Status', 'yes', 'year', 'amount', 'log_amount', )
            )

# County Data

In [ ]:
landvote_county = landvote_df.filter(_.jurisdiction == "County")

county_boundaries = "https://data.source.coop/cboettig/us-boundaries/us-county.parquet"

df_county = (conn
            .read_parquet(county_boundaries)
            .rename(county = "NAMELSAD", state_name = "STATE_NAME")
            .join(state, "state_name", how = "inner")
            .select('state','state_name','county','geometry')
            .cast({"geometry": "geometry"})
            .join(landvote_county, ["county","state"],how = "inner")
            .select('state','county','municipal','jurisdiction','geometry','Status', 'yes', 'year', 'amount', 'log_amount', )
             )

# Municipal Data

In [ ]:
landvote_city = landvote_df.filter(_.jurisdiction == "Municipal")

df_city = (conn
            .read_parquet("us_localities.parquet")
            .select('state','county','municipal','geometry')
            .cast({"geometry": "geometry"})
            .join(landvote_city, ["municipal","state"], how = "inner")
            .select('state','county','municipal','jurisdiction','geometry','Status', 'yes', 'year', 'amount', 'log_amount', )
            )

# Make PMTiles. Each jurisdiction type is its own layer

In [ ]:
import subprocess
import os
from huggingface_hub import HfApi, login
import streamlit as st

login(st.secrets["HF_TOKEN"])
# api = HfApi(add_to_git_credential=False)
api = HfApi()

def hf_upload(file, repo_id):
    info = api.upload_file(
            path_or_fileobj=file,
            path_in_repo=file,
            repo_id=repo_id,
            repo_type="dataset",
        )
def generate_pmtiles(input_file, input_file2, input_file3, output_file, max_zoom=12):
    # Ensure Tippecanoe is installed
    if subprocess.call(["which", "tippecanoe"], stdout=subprocess.DEVNULL) != 0:
        raise RuntimeError("Tippecanoe is not installed or not in PATH")

    # Construct the Tippecanoe command
    command = [
        "tippecanoe",
        "-o", output_file,
        "-zg",
        "--extend-zooms-if-still-dropping",
        "--force",
        "--projection", "EPSG:4326",  
        "-L","state:"+input_file,
        "-L","county:"+input_file2,
        "-L","municipal:"+input_file3
    ]
    # Run Tippecanoe
    try:
        subprocess.run(command, check=True)
        print(f"Successfully generated PMTiles file: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error running Tippecanoe: {e}")



In [ ]:
gdf_state = df_state.execute().set_crs("EPSG:4326")
gdf_state.to_file("landvote_state.geojson")

gdf_county = df_county.execute().set_crs("EPSG:4326")
gdf_county.to_file("landvote_county.geojson")

gdf_city = df_city.execute().set_crs("EPSG:4326")
gdf_city.to_file("landvote_municipal.geojson")

generate_pmtiles("landvote_state.geojson", "landvote_county.geojson","landvote_municipal.geojson", "landvote_polygons.pmtiles")
hf_upload("landvote_polygons.pmtiles", "boettiger-lab/landvote")


# Combine all 3 jurisdiction types into a parquet.

In [ ]:
df_temp = df_county.union(df_city)
df = df_temp.union(df_state)
df.execute().set_crs("EPSG:4326").to_parquet("landvote_polygons.parquet")
hf_upload("landvote_polygons.parquet", "boettiger-lab/landvote")
